In [ ]:
# hide
#default_exp receipt
#default_cls_lvl 3
from nbdev import *
from drt.receipt import *
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
#export
from pathlib import Path
import drt.data_model as dm
import re
from typing import Union
from drt.environment import DataIntakeEnv
from dateutil import parser
from sqlalchemy import inspect
from typing import Union
from drt.utils import Data_Groups_Type

# Receipt

> Generate ReST formatted data stubs for each data element which can be used as a base for a statically generated site and/or updated by users to correct information. In particular, these receipts are where analysts will be able to enter details about the data elements.

In [ ]:
#export
def sync_data_folder(env: DataIntakeEnv, data_group_type: Data_Groups_Type):
    """
    TODO
    [summary]

    ##### Parameters
    env : DataIntakeEnv
        [description]

    data_group_type : Data_Groups_Type
        [description]

    ##### Raises  
    TypeError
        [description]

    """
    if data_group_type == dm.Delivery:
        data_folder = env.delivery_folder
    elif data_group_type == dm.Raw_Data:
        data_folder = env.raw_data_folder
    elif data_group_type == dm.Dataset:
        data_folder = env.dataset_folder
    else:
        raise TypeError(data_group_type)

    for data_group in data_folder.iterdir():
        sync_data_group(env, data_group)

In [ ]:
#export
def sync_data_group(env: DataIntakeEnv, data_group: Path):
    """
    TODO
    [summary]

    ##### Parameters
    env : [type]
        [description]

    data_group : Path
        [description]

    data_group_type : str
        [description]


    ##### Raises
    TypeError
        [description]

    """

    data_group_type = env.get_group_type_from_path(data_group)

    if data_group.is_dir() and (data_group / 'receipt.rst').exists():
        record = env.session.query(data_group_type).filter_by(name=data_group.name).first()
        if record:
            sync_receipt(env, (data_group / 'receipt.rst'), record)
        else:
            print(f'[!] record not found for {data_group.name}, can''t sync')

In [ ]:
#export
def sync_receipt(env: DataIntakeEnv, receipt_path: Path, data_group: Data_Groups_Type):
    """
    TODO
    [summary]

    ##### Parameters 
    env : DataIntakeEnv
        [description]

    receipt_path : Path
        [description]

    data_group : Union[dm.Dataset, dm.Delivery, dm.Raw_Data]
        [description]


    ##### Raises
    TypeError
        [description]

    """

    receipt_data = parse_receipt(env, receipt_path)

    for k, v in receipt_data.items():
        if k == 'description':
            data_group.description = v
        elif k == 'date_received':
            data_group.date_received = v
        elif k == 'delivery_source':
            data_group.delivery_source = v
        elif k == 'statistics_report':
            data_group.statistics_report = v
        elif k == 'dataset_report':
            data_group.dataset_report = v

    env.session.commit()

    # for each column compare text repr with data
    # cols have different masters:
    #  [[done]] Description, Source, Received date, are file master
    #  Tags are file master
    #  [[done]] System fields are db master
    #  links are outer join of file and db

    # update DB
    print(f"Writing receipt to {receipt_path.parent}")
    write_receipt(env, receipt_path.parent)

In [ ]:
#export
def write_receipt(env: DataIntakeEnv, folder: Path):
    """
    Create a receipt for a data group based on information from the sqlite database.

    ##### Parameters 
    env : DataIntakeEnv
        The data registration environment

    folder : Path
        Folder path to the Data Group to create a receipt for.

    group_type : str
        What type of data group is this data element.


    ##### Raises
    TypeError
        Raised if the data group type is not correct

    """

    dg = env.get_group_type_from_path(folder)

    # Get folder info from database
    folder_info = env.session.query(dg).filter_by(name=folder.name).first()

    # Write data to folder using data model serialization
    with open((folder / 'receipt.rst'), mode='wt') as f:
        try:
            f.write(folder_info.document())
        except Exception as e:
            print("Registering Failed")
            print(folder.name, folder_info, dg)
            raise e

In [ ]:
#export
def parse_receipt(env: DataIntakeEnv, receipt_path: Path) -> dict:
    """
    TODO
    [summary]

    ##### Parameters
    receipt_path : Path
        [description]

    group_type : str
        [description]


    ##### Returns
    dict
        [description]

    ##### Raises
    TypeError
        [description]

    """
    text = receipt_path.read_text()

    group_type = env.get_group_type_from_path(receipt_path.parent)

    patterns = dict()
    data = dict()

    if group_type == dm.Delivery:
        # extract data with these regexes
        patterns['description'] = re.compile(r'Description:\n-+\n(.*?)\n+[^\n]+\n-+\n', re.MULTILINE | re.DOTALL)
        patterns['date_received'] = re.compile(r'\n:Date Received: (.+)$', re.MULTILINE)
        patterns['delivery_source'] = re.compile(r'\n:Received from: (.+)$', re.MULTILINE)
    elif group_type == dm.Raw_Data:
        # extract data with these regexes
        patterns['description'] = re.compile(r'Description:\n-+\n(.*?)\n+[^\n]+\n-+\n', re.MULTILINE | re.DOTALL)
        patterns['statistics_report'] = re.compile(r'Report\n-+\n(.*?)\n.*', re.MULTILINE | re.DOTALL)
        pass
    elif group_type == dm.Dataset:
        # extract data with these regexes
        patterns['description'] = re.compile(r'Description:\n-+\n(.*?)\n+[^\n]+\n-+\n', re.MULTILINE | re.DOTALL)
        patterns['dataset_report'] = re.compile(r'Report\n-+\n(.*?)\n.*', re.MULTILINE | re.DOTALL)
        pass
    else:
        raise TypeError

    for k, v in patterns.items():
        res = re.search(v, text)
        if res:
            data[k] = res.group(1)
            if data[k] == 'None':
                data[k] = None
            if 'date' in k and not data[k] is None:
                data[k] = parser.parse(data[k])

    return data

In [ ]:
import re 

text = '''


File Metadata
-------------
:Files Last Changed: 2021-02-10 20:32:52.637435
:File Size: 1244762
:Number of Files: 3
:Hash: 975bd61edb1adec7b61b8fb459de04e775d286bcc5cd19a4be67c74c04044e46

Report
------
Dummy HTML Report


'''
re.search(r'Report\n-+\n(.*?)\n.*',text,  re.MULTILINE | re.DOTALL)[1]

'Dummy HTML Report'

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_tests.ipynb.
Converted 01_cli.ipynb.
Converted 02_create_environment.ipynb.
Converted 03_data_model.ipynb.
Converted 04_environment.ipynb.
Converted 05_receipt.ipynb.
Converted 06_registration.ipynb.
Converted 07_verification.ipynb.
Converted 99_utils.ipynb.
Converted index.ipynb.


[autoreload of drt.data_model failed: Traceback (most recent call last):
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 292, in update_class
    if (old_obj == new_obj) is True:
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/sqlalchemy/sql/operators.py", line 365, in __eq__
    return self.operate(eq, other)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/sqlalchemy/orm/attributes.py", line 226, in operate
    return op(self.comparator, *other, **kwargs)
 